trial and draft file

import statements

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

load the data files

In [2]:
here = os.getcwd()

#loads gps-location name correspondance file
afilename = here+"/data/2020/map-keys-2020.csv"
places = pd.read_csv(afilename)

#loads species data set
afilename = here+"/data/2020/species2020_full.csv"
samples = pd.read_csv(afilename)

date data

In [3]:
#check date data for missing values and converts to strptime format.
def check_dates(x):
    try:
        data = dt.datetime.strptime(x, "%Y-%m-%d")
    except:
        data = 'no date'
    return data
samples['date_stamp'] = samples['date'].map(lambda x: check_dates(x))

#looks at monthly and daily collections
samples['month'] = samples.date_stamp.dt.month

print(samples.date.describe())
print("the unique months are:")
print(samples.month.unique())
june = samples.loc[date[0]]
print(june)

count            935
unique            65
top       2020-09-02
freq              55
Name: date, dtype: object
the unique months are:
[9 7 6 8]


NameError: name 'date' is not defined